# Get started with `fused-py`

Integrate Fused into your own environment with the Fused Python SDK (`fused-py`). The library exposes utility functions to write, manage, and run Fused UDFs from anywhere. 

This guide provides a setup overview and a brief introduction to get started with `fused-py` in a Jupyter Notebook.

## Setup

### 1. Create an account at fused.io
...


### 2. Install the Python package

Start by installing [`fused-py`](https://pypi.org/project/fused/).

:::note

Fused supports Python versions >=3.8 to <3.12.

:::



In [ ]:
# !pip install fused -q

### 3. Authenticate

The first time you run Fused in a Notebook, you may be prompted to authenticate. To authenticate you'll need to have [created a Fused account](/workbench/get-started/#1-sign-up-for-fused), although it's also possible to [run UDFs without an account](/python-sdk/get-started/#run-udfs-without-an-account).

Call `NotebookCredentials` in a cell and follow the authentication flow.



In [ ]:
from fused.api import NotebookCredentials
credentials = NotebookCredentials()

Fused will write credentials to disk (in `~/.fused/credentials`) and `fused-py` will have access to the Fused API.


:::info

When running code from a Python environment without browser access, you may copy the `~/.fused/credentials` file to the same location in the target system.

:::


## Quickstart

### 1. Define a UDF

Fused takes your Python code and runs in in the cloud.

Let's write a simple function that takes an input, prints a log to the console, and then returns an output.

To make this function a Fused UDF, simply decorate it with `@fused.udf`. 

In [1]:
import fused

@fused.udf
def udf(message):
    import pandas as pd
    return pd.DataFrame({'a': [message]})

### 2. Run a UDF locally and remotely

You run UDFs with `fused.run`. There are two ways to run the UDF:
1. As a regular function call on your local machine, with `engine='local'`
2. As a remote function call on the Fused cloud, with `engine='realtime'`


In [ ]:
# Run locally
fused.run(udf, message='Hello, world!', engine='local')

# Run on Fused
fused.run(udf, message='Hello, world!', engine='realtime')

#### What just happened?
When you called `fused.run` with the realtime engine, the function was executed on Fused's cloud infrastructure. Fused took the function, ran it in a cloud instance, and returned the output. 

The goal of Fused is to make running Python in the cloud feel like you're running it locally - with added benefits such as easy autoscaling and no need to manage infrastructure.

### 3. Load and run a UDF

You can load UDFs from various sources: GitHub repositories, local files, and your Fused account. This fosters discoverability within teams and the UDF community, promotes reuse of existing code, and simplifies your workflows.

Load a UDF from a GitHub URL and inspect its repr:

In [ ]:
udf = fused.load("https://github.com/fusedio/udfs/tree/main/public/DuckDB_NYC_Example")
udf

Run the loaded UDF:

In [ ]:
gdf = fused.run(udf, engine='local')
gdf.head()

Run a Public UDF by its name:

In [ ]:
gdf = fused.run("UDF_DuckDB_NYC_Example", engine='realtime')
gdf.head()

:::note

Loading UDFs from GitHub repositories or local files does not require authentication to the Fused platform.

:::



Run a Public UDF by its name, and pass a custom parameter value:

In [ ]:
import geopandas as gpd
import fused

bbox = gpd.GeoDataFrame.from_features({"type": "FeatureCollection","features": [{"type": "Feature","properties": {},"geometry": {"coordinates": [[[3.390159017607914,6.463364738709416],[3.386024182498403,6.460179445052702],[3.3878593059455397,6.457224951853391],[3.3933182174667422,6.457132623662659],[3.3933182174667422,6.461541275955156],[3.390159017607914,6.463364738709416]]],"type": "Polygon"}}]})

# gdf = fused.run("UDF_Overture_Maps_Example", bbox=(-77.34375, 38.41055, -77.167968, 38.54816), engine='realtime')
gdf = fused.run("UDF_Overture_Maps_Example", bbox=[-77.34375, 38.41055, -77.167968, 38.54816], engine='realtime')
# gdf.plot()
gdf.head(1)

In [ ]:
udf = fused.load("https://github.com/fusedio/udfs/tree/main/public/Overture_Maps_Example")
gdf = fused.run(udf, bbox=bbox, engine='realtime')
gdf.head(1)

## 4. Save a UDF 



In [ ]:
udf = fused.load("https://github.com/fusedio/udfs/tree/main/public/Overture_Maps_Example")
udf.name="My_Oveture_UDF"
udf.to_fused()

## 5. List udfs in your account

In [ ]:
all_udfs = fused.api.get_udfs(n=100)
fused.run(all_udfs.DuckDB_NYC_Example)

In [ ]:
# by name in my account
fused.run("coffee_narwhal", engine='realtime')

In [ ]:
bbox

In [ ]:
fused.run(all_udfs.DuckDB_NYC_Example)

In [ ]:
@fused.udf
def udf2(message):
    import pandas as pd
    return pd.DataFrame({'a': [message]})
udf2.to_fused()


## 6. create endpoint






In [5]:
from fused.api import FusedAPI


api = FusedAPI()
token = api.create_udf_access_token('coffee_narwhal')
token

UdfAccessToken(token='fsh_7b0h8q7gagf51TuMqsLdPt', udf_email='plinio@fused.io', udf_slug='coffee_narwhal', udf_id='5522fc4a-970c-4967-b5ac-cb0316f2de21', enabled=True, owning_user_id='20a95eb8-a25b-4f1f-924c-80a41c70b796', client_id='fused', cache=True, metadata_json={}, last_updated=datetime.datetime(2024, 10, 1, 19, 59, 30, 364586, tzinfo=TzInfo(UTC)))

In [20]:
# !pip3 install leafmap -q

In [22]:
url = token.get_tile_url() + "?dtype_out_vector=mvt"
url

'https://www.fused.io/server/v1/realtime-shared/fsh_7b0h8q7gagf51TuMqsLdPt/run/tiles/{z}/{x}/{y}?dtype_out_vector=mvt'

In [ ]:
import geemap
Map = geemap.Map()
from ipyleaflet import GeoJSON, Polyline, TileLayer

# Geemap VectorTileLayer
dark_matter_layer = VectorTileLayer(url='https://www.fused.io/server/v1/realtime-shared/fsh_6CK4ljxHzP2FB6KxhZTU0H/run/tiles/{z}/{x}/{y}?dtype_out_vector=mvt')
Map.add_layer(dark_matter_layer)


In [23]:
import leafmap.foliumap as leafmap
m = leafmap.Map(center=(52.204793, 360.121558), zoom=9)
m.add_vector_tile_layer(url)
m

In [ ]:
import fused
fused.api.get_udf_access_token
# fused.api._check_is_prod()

## 7. plot in map xyz

In [ ]:
import fused
import geopandas as gpd

udf = fused.load("https://github.com/fusedio/udfs/tree/main/public/Overture_Maps_Example")
gdf_output = fused.run(udf, x=2622, y=6333, z=14, engine='local')
